# Python-tasks

Python task definitions are Python functions that are parameterised in a separate step before
they are executed or added to a workflow.

## Define decorator

The simplest way to define a Python task is to decorate a function with `pydra.design.python.define`

In [1]:
from pydra.design import python

# Note that we use CamelCase as the return of the is a class
@python.define
def MyFirstTaskDef(a, b):
    """Sample function for testing"""
    return a + b

The resulting task-definition class can be then parameterized (instantiated), and
executed

In [ ]:
# Instantiate the task, setting all parameters
my_first_task = MyFirstTaskDef(a=1, b=2.0)

# Execute the task
outputs = my_first_task()

print(outputs.out)

A newer version (0.25) of nipype/pydra is available. You are using 0.25.dev103+g1a6b067c.d20241228


3.0


By default, the name of the output field for a function with only one output is `out`. To
name this something else, in the case where there are multiple output fields, the `outputs`
argument can be provided to `python.define`


In [ ]:
@python.define(outputs=["c", "d"])
def NamedOutputTaskDef(a, b):
    """Sample function for testing"""
    return a + b, a - b

named_output_task = NamedOutputTaskDef(a=2, b=1)

outputs = named_output_task()

print(outputs)

NamedOutputTaskOutputs(c=3, d=1)


The input and output field attributes automatically extracted from the function, explicit
attributes can be augmented

In [8]:
@python.define(
    inputs={"a": python.arg(allowed_values=[1, 2, 3]), "b": python.arg(default=10.0)},
    outputs={
        "c": python.out(type=float, help_string="the sum of the inputs"),
        "d": python.out(type=float, help_string="the difference of the inputs"),
    },
)
def AugmentedTaskDef(a, b):
    """Sample function for testing"""
    return a + b, a - b

## Type annotations

If provided, type annotations are included in the task definition, and are checked at
the time of parameterisation.

In [4]:
from pydra.design import python

# Note that we use CamelCase as the function is translated to a class

@python.define
def MyTypedTask(a: int, b: float) -> float:
    """Sample function for testing"""
    return a + b

try:
    # 1.5 is not an integer so this should raise a TypeError
    my_typed_task = MyTypedTask(a=1.5, b=2.0)
except TypeError as e:
    print(f"Type error caught: {e}")
else:
    assert False, "Expected a TypeError"

# While 2 is an integer, it can be implicitly coerced to a float
my_typed_task = MyTypedTask(a=1, b=2)

Type error caught: Incorrect type for field in 'a' field of MyTypedTask interface : 1.5 is not of type <class 'int'> (and cannot be coerced to it)



## Docstring parsing

Instead of explicitly providing help strings and output names in `inputs` and `outputs`
arguments, if the function describes the its inputs and/or outputs in the doc string, 
in either reST, Google or NumpyDoc style, then they will be extracted and included in the
input or output fields


In [12]:
from pprint import pprint
from pydra.engine.helpers import fields_dict

@python.define
def DocStrDef(a: int, b: float) -> tuple[float, float]:
    """Sample function for testing

    Args:
        a: First input
            to be inputted
        b: Second input

    Returns:
        c: Sum of a and b
        d: Product of a and b
    """
    return a + b, a * b

pprint(fields_dict(DocStrDef))
pprint(fields_dict(DocStrDef.Outputs))

{'a': arg(name='a', type=<class 'int'>, default=EMPTY, help_string='First input to be inputted', requires=[], converter=None, validator=None, allowed_values=(), xor=(), copy_mode=<CopyMode.any: 15>, copy_collation=<CopyCollation.any: 0>, copy_ext_decomp=<ExtensionDecomposition.single: 1>, readonly=False),
 'b': arg(name='b', type=<class 'float'>, default=EMPTY, help_string='Second input', requires=[], converter=None, validator=None, allowed_values=(), xor=(), copy_mode=<CopyMode.any: 15>, copy_collation=<CopyCollation.any: 0>, copy_ext_decomp=<ExtensionDecomposition.single: 1>, readonly=False),
 'function': arg(name='function', type=typing.Callable, default=<function SampleDef at 0x10d0253a0>, help_string='', requires=[], converter=None, validator=None, allowed_values=(), xor=(), copy_mode=<CopyMode.any: 15>, copy_collation=<CopyCollation.any: 0>, copy_ext_decomp=<ExtensionDecomposition.single: 1>, readonly=False)}
{'c': out(name='c', type=<class 'float'>, default=EMPTY, help_string='S

## Wrapping external functions

Like all decorators, `python.define` is just a function, so can also be used to convert
a function that is defined separately into a Python task definition.

In [5]:
import numpy as np

NumpyCorrelate = python.define(np.correlate)

numpy_correlate = NumpyCorrelate(a=[1, 2, 3], v=[0, 1, 0.5])

outputs = numpy_correlate()

print(outputs.out)

[3.5]


Like with decorated functions, input and output fields can be explicitly augmented via
the `inputs` and `outputs` arguments

In [7]:
import numpy as np

NumpyCorrelate = python.define(np.correlate, outputs=["correlation"])

numpy_correlate = NumpyCorrelate(a=[1, 2, 3], v=[0, 1, 0.5])

outputs = numpy_correlate()

print(outputs.correlation)

[3.5]
